## Load Model

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread
#Download model
!ollama pull mathstral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 9bf04412c5e2...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling 9bf04412c5e2...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling 9bf04412c5e2...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling 9bf04412c5e2...   1% ▕                ▏  42 MB/4.1 GB                  pulling manifest 
pulling 9bf04412c5e2...   3% ▕                ▏ 120 MB/4.1 GB                  pulling manifest 
pulling 9bf04412c5e2...   4% ▕                ▏ 156 MB/4.1 GB                  pulling manifest 
pulling 9bf04412c5e2...   6% ▕                ▏ 250 MB/4.1 GB                  pulling manifest 
pulling 9bf04412c5e2...   8% ▕█               ▏ 329 MB/4.1 GB                  pulling manifest 
pulling 9bf04412c5e2...   9% ▕█    

In [3]:
!pip install ollama

## Load Dataset

In [ ]:
import time
import copy
import os
import pandas as pd
import ollama

def load_dataset(task):
    """
    Load dataset for a given task.
    """
    task1 = pd.read_csv("/kaggle/input/BenNumEval_run_1_random_1000/Task1_random_150.csv")
    task2 = pd.read_csv("/kaggle/input/BenNumEval_run_1_random_1000/Task2_random_250.csv")
    task3 = pd.read_csv("/kaggle/input/BenNumEval_run_1_random_1000/Task3_random_150.csv")
    task4 = pd.read_csv("/kaggle/input/BenNumEval_run_1_random_1000/Task4_random_150.csv")
    task5 = pd.read_csv("/kaggle/input/BenNumEval_run_1_random_1000/Task5_random_150.csv")
    task6 = pd.read_csv("/kaggle/input/BenNumEval_run_1_random_1000/Task6_random_150.csv")

    task_dict = {
        "task1": task1,
        "task2": task2,
        "task3": task3,
        "task4": task4,
        "task5": task5,
        "task6": task6
    }
    return task_dict.get(task, "Invalid task")

## Prompts

In [ ]:
# Prompt templates
xlp_prompt_template_task1246 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a mathematical reasoning problem provided in Bengali and deliver the solution in English.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate the Bengali problem into English for clarity.\n"
    "2. Conclude with the final numerical answer, formatted as **\"Answer: [num]\"**.\n\n"
    "# Problem: {question}\n\n"
    "# Response:"
)

xcot_prompt_template_task1246 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a mathematical reasoning problem provided in Bengali and deliver the solution in English.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate the Bengali problem into English for clarity.\n"
    "2. Provide a detailed step-by-step solution with explanations in English, following the logical flow of reasoning.\n"
    "3. Conclude with the final numerical answer, formatted as **\"Answer: [num]\"**.\n\n"
    "# Problem: {question}\n\n"
    "# Response:"
)

bnap_prompt_template_task1246 = (
    "# নির্দেশাবলী:\n"
    "আপনি একজন গণিত বিশেষজ্ঞ এআই মডেল, যিনি বাংলা ভাষায় সম্পূর্ণভাবে দক্ষ। আপনার কাজ হলো প্রদত্ত গাণিতিক সমস্যার বিশদভাবে সমাধান করা এবং উত্তরটি বাংলায় প্রদান করা।\n"
    "আপনাকে অবশ্যই চূড়ান্ত সাংখ্যিক উত্তরটি নিম্নলিখিত ফরম্যাটে উপস্থাপন করতে হবে: **\"উত্তর: [সংখ্যা]\"**।\n\n"
    "# সমস্যা: {question}\n\n"
    "# সমাধান:"
)

xlp_prompt_template_task3 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a mathematical reasoning problem provided in Bengali and choose the correct option.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate the Bengali question and options into English for clarity.\n"
    "2. Conclude by selecting the correct option, formatted as **\"Answer: [Option]\"**. The possible options are **\"Option 1\"** or **\"Option 2\"**.\n\n"
    "# Question: {question}\n"
    "# Option 1: {option1}\n"
    "# Option 2: {option2}\n\n"
    "# Response:"
)

xcot_prompt_template_task3 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a mathematical reasoning problem provided in Bengali and choose the correct option.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate the Bengali question and options into English for clarity.\n"
    "2. Provide a detailed step-by-step solution with explanations in English, following the logical flow of reasoning.\n"
    "3. Conclude by selecting the correct option, formatted as **\"Answer: [Option]\"**. The possible options are **\"Option 1\"** or **\"Option 2\"**.\n\n"
    "# Question: {question}\n"
    "# Option 1: {option1}\n"
    "# Option 2: {option2}\n\n"
    "# Response:"
)


bnap_prompt_template_task3 = (
    "# নির্দেশাবলী:\n"
    "আপনি একজন দক্ষ গণিত বিশেষজ্ঞ এআই, যিনি বাংলা ভাষায় পারদর্শী। আপনার কাজ হলো প্রদত্ত গাণিতিক সমস্যার বিশ্লেষণ করে সঠিক উত্তর নির্বাচন করা।\n\n"
    "আপনার উত্তর অবশ্যই *নির্দিষ্ট বিন্যাসে* প্রদান করতে হবে:\n"
    "**\"উত্তর: [সঠিক সম্ভাব্য উত্তর]\"**\n" 
    "যেখানে **[সঠিক সম্ভাব্য উত্তর]** হবে **\"উত্তর ১\"** অথবা **\"উত্তর ২\"**।\n\n"
    "---\n" 
    "# সমস্যা: {question}\n"
    "# সম্ভাব্য উত্তরসমূহ:\n"
    "# সম্ভাব্য উত্তর ১: {option1}\n"
    "# সম্ভাব্য উত্তর ২: {option2}\n"  
    "---\n\n" 
    "# সমাধান:"
    )


xlp_prompt_template_task5 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a Quantitative Natural Language Inference (QNLI) problem presented in Bengali. You need to determine the relationship between the premise and the hypothesis.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate both the premise and hypothesis from Bengali to English for clarity.\n"
    "2. Conclude by selecting the correct option, formatted as **\"Answer: [Option]\"**. The possible options are **\"Entailment\"**, **\"Neutral\"**, or **\"Contradiction\"**.\n\n"
    "# Premise: {premise}\n"
    "# Hypothesis: {hypothesis}\n\n"
    "# Response:"
)

xcot_prompt_template_task5 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a Quantitative Natural Language Inference (QNLI) problem presented in Bengali. You need to determine the relationship between the premise and the hypothesis.\n\n"
    "Please follow these steps:\n"
    "1. Translate both the premise and hypothesis from Bengali to English for clarity.\n"
    "2. Provide a step-by-step explanation of your reasoning process.\n"
    "3. Conclude by selecting the correct option, formatted as **\"Answer: [Option]\"**. The possible options are **\"Entailment\"**, **\"Neutral\"**, or **\"Contradiction\"**.\n\n"
    "# Premise: {premise}\n"
    "# Hypothesis: {hypothesis}\n\n"
    "# Response:"
)

bnap_prompt_template_task5 = (
    "# নির্দেশাবলী:\n"
    "আপনি একজন গণিত বিশেষজ্ঞ AI মডেল, যিনি বাংলা ভাষায় দক্ষ। আপনার কাজ হলো একটি গাণিতিক ভাষাগত অনুমান সমস্যার সমাধান করা। আপনাকে প্রদত্ত পূর্বধারণা ও অনুমান এর মধ্যে সম্পর্ক নির্ধারণ করতে হবে।\n\n"
    "অনুগ্রহ করে নিম্নলিখিত বিন্যাস অনুসরণ করুন:\n"
    "প্রদত্ত পূর্বধারণা ও অনুমান এর মধ্যে সম্পর্ক বিষয়ে চূড়ান্ত সিদ্ধান্ত নিন এবং সঠিক উত্তরটি **\"উত্তর: [সম্ভাব্য সঠিক উত্তর]\"** এই ভাবে প্রদান করুন। উত্তরের সম্ভাব্য বিকল্পগুলি হল: **\"সমর্থন\"**, **\"নিরপেক্ষ\"**, অথবা **\"বিরোধ\"**।\n\n"
    "# পূর্বধারণা: {premise}\n"
    "# অনুমান: {hypothesis}\n\n"
    "# সমাধান:"
)

# Map prompt template names to their content
prompt_map = {
    "xlp_prompt_template_task1246": xlp_prompt_template_task1246,
    "xcot_prompt_template_task1246": xcot_prompt_template_task1246,
    "bnap_prompt_template_task1246": bnap_prompt_template_task1246,
    "xlp_prompt_template_task3": xlp_prompt_template_task3,
    "xcot_prompt_template_task3": xcot_prompt_template_task3,
    "bnap_prompt_template_task3": bnap_prompt_template_task3,
    "xlp_prompt_template_task5": xlp_prompt_template_task5,
    "xcot_prompt_template_task5": xcot_prompt_template_task5,
    "bnap_prompt_template_task5": bnap_prompt_template_task5,
}

## Inference

In [ ]:
prompts = ["xlp", "xcot", "bnap"]
model_name = "Mathstral_7B"

def process_prompts_for_task(task_id, data):
    """
    Process a given task with the provided dataset and for each prompt type.
    """
    for prompt in prompts:
        responses = []
        chat_histories = []
        num_instances = len(data)
        print(f"Processing Task {task_id} with prompt {prompt} for {num_instances} instances...")

        for i in range(num_instances):
            # Prepare variables for formatting prompt based on task type
            if task_id in [1, 2, 4, 6]:
                question = data.iloc[i]['Question']
                # template_key = f"{prompt}_prompt_template_task1246"
                # template = prompt_map.get(template_key)
                # formatted_prompt = template.format(question=question)
                formatted_prompt = prompt_map.get(f"{prompt}_prompt_template_task1246").format(question=question)
            elif task_id == 3:
                question = data.iloc[i]['Question']
                option1 = data.iloc[i]['option1']
                option2 = data.iloc[i]['option2']
                formatted_prompt = prompt_map.get(f"{prompt}_prompt_template_task{task_id}").format(
                    question=question, option1=option1, option2=option2
                )
            elif task_id == 5:
                # For Task 5, data contains 'Premise' and 'Hypothesis'
                premise = data.iloc[i]['Premise']
                hypothesis = data.iloc[i]['Hypothesis']
                formatted_prompt = prompt_map.get(f"{prompt}_prompt_template_task{task_id}").format(
                    premise=premise, hypothesis=hypothesis
                )
            else:
                print(f"Invalid task id: {task_id}")
                continue

            # Start a new chat session and send the formatted prompt
            response = ollama.chat(
                        model='mathstral',
                        messages=[
                            {
                                'role': 'user',
                                'content': formatted_prompt,
                            },
                        ],
                        options={
                            'temperature': 1.0,
                            'top_p': 1.0,
                            'num_predict': 512
                        }
                    )
            responses.append(response.message.content)
            chat_histories.append(response)

            print(f"\033[91mTask {task_id} | Prompt {prompt} | Question No: {i}\033[0m")
        
        # Prepare the dataframe to save the responses and chat histories
        data_with_solutions = copy.deepcopy(data)
        data_with_solutions["Model Response"] = responses
        data_with_solutions["Chat History"] = chat_histories
        
        # Save the responses to a CSV file
        output_filename = f"{model_name}_{prompt}_task{task_id}_random_{num_instances}_responses.csv"
        data_with_solutions.to_csv(output_filename, index=False)
        print(f"Saved responses to {output_filename}")
        
        # Optionally, print a sample response (if exists)
        print(data_with_solutions.iloc[10]["Model Response"])
        
        # print("Sleeping for 5 seconds before next prompt...")
        # time.sleep(5)
    
    print("Completed all prompts for Task", task_id)
    # print("Sleeping for 5 seconds before next task...")
    # time.sleep(5)

In [7]:
def process_task(task_id):
    """
    Load dataset for the task and process based on task type.
    """
    dataset_key = f"task{task_id}"
    data = load_dataset(dataset_key)
    if isinstance(data, str):
        print(f"Error loading dataset for task: {task_id}")
        return

    # For tasks 1, 2, 4, 6, 3, and 5 the processing is handled inside process_prompts_for_task.
    process_prompts_for_task(task_id, data)

def main():
    for task in [1, 4, 3, 5, 6, 2]:
        print(f"Starting processing for Task {task}")
        process_task(task)
    print("All tasks processed.")

if __name__ == "__main__":
    main()

Starting processing for Task 1
Processing Task 1 with prompt xlp for 150 instances...
Task 1 | Prompt xlp | Question No: 0
Task 1 | Prompt xlp | Question No: 1
Task 1 | Prompt xlp | Question No: 2
Task 1 | Prompt xlp | Question No: 3
Task 1 | Prompt xlp | Question No: 4
Task 1 | Prompt xlp | Question No: 5
Task 1 | Prompt xlp | Question No: 6
Task 1 | Prompt xlp | Question No: 7
Task 1 | Prompt xlp | Question No: 8
Task 1 | Prompt xlp | Question No: 9
Task 1 | Prompt xlp | Question No: 10
Task 1 | Prompt xlp | Question No: 11
Task 1 | Prompt xlp | Question No: 12
Task 1 | Prompt xlp | Question No: 13
Task 1 | Prompt xlp | Question No: 14
Task 1 | Prompt xlp | Question No: 15
Task 1 | Prompt xlp | Question No: 16
Task 1 | Prompt xlp | Question No: 17
Task 1 | Prompt xlp | Question No: 18
Task 1 | Prompt xlp | Question No: 19
Task 1 | Prompt xlp | Question No: 20
Task 1 | Prompt xlp | Question No: 21
Task 1 | Prompt xlp | Question No: 22
Task 1 | Prompt xlp | Question No: 23
Task 1 | Pro